In [1]:
# Load the pickled data
import pickle

file_location = 'D:\\5005-Data\\tweet_combined_with_sentiment.pkl'

with open(file_location, 'rb') as f:
    tweets_dict = pickle.load(f)

len(tweets_dict)

1387322

In [18]:
import numpy as np

like_ratios = []
for k, tweet in tweets_dict.items():
    if tweet.viewCount is None:
        continue
    if tweet.lang != 'en':
        continue
    if tweet.inReplyToUser is not None:
        continue
    ratio = tweet.likeCount / tweet.viewCount
    like_ratios.append(ratio)

p50 = np.percentile(like_ratios, 50)
p90 = np.percentile(like_ratios, 90)

print(len(like_ratios))
print(p50, p90)

776029
0.0 0.02564102564102564


In [23]:
import re

def remove_urls_and_entities(text):
    """
    Removes URLs and HTML entities from a string using regular expressions.
    
    Args:
        text (str): The input string to remove URLs and HTML entities from.
        
    Returns:
        str: The input string with any URLs and HTML entities removed.
    """
    # Define regular expressions to match URLs and HTML entities
    url_pattern = re.compile(r'https?://(?:www\.\S+|(?!www)\S+)')
    entity_pattern = re.compile(r'&\w+;')
    
    # Use the sub() method to replace URLs and HTML entities with an empty string
    text_without_urls_and_entities = url_pattern.sub('', text)
    text_without_urls_and_entities = entity_pattern.sub('', text_without_urls_and_entities)
    text_without_urls_and_entities = text_without_urls_and_entities.replace('\n', ' ')
    
    return text_without_urls_and_entities

In [38]:
lines = []

for k, tweet in tweets_dict.items():
    if tweet.viewCount is None:
        continue
    if tweet.lang != 'en':
        continue
    if tweet.inReplyToUser is not None:
        continue
    content = remove_urls_and_entities(tweet.rawContent)
    label = 0
    if tweet.likeCount / tweet.viewCount > 0:
        label = 1
    if tweet.likeCount / tweet.viewCount > 0.025:
        label = 2
    lines.append('__label__' + str(label) + ' ' + content)

part = int(len(lines) * 0.1)

f = open('train.ftxt', 'w', encoding='utf-8')
train_lines = lines[:part * 8]
f.write('\n'.join(train_lines))
f.close()

f = open('test.ftxt', 'w', encoding='utf-8')
test_lines = lines[part * 8:part * 9]
f.write('\n'.join(test_lines))
f.close()

f = open('dev.ftxt', 'w', encoding='utf-8')
test_lines = lines[part * 9:]
f.write('\n'.join(test_lines))
f.close()

In [2]:
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = ClassificationCorpus(Path('./'), test_file='test.ftxt', dev_file='dev.ftxt', train_file='train.ftxt')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_type='class', label_dictionary=corpus.make_label_dictionary('class'))


d:\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2023-04-02 19:05:53,680 Reading data from .
2023-04-02 19:05:53,680 Train: train.ftxt
2023-04-02 19:05:53,680 Dev: dev.ftxt
2023-04-02 19:05:53,681 Test: test.ftxt
2023-04-02 19:06:13,735 Initialized corpus . (label type name is 'class')
2023-04-02 19:06:16,628 Computing label dictionary. Progress:


C:\Users\user\AppData\Local\Temp\ipykernel_9248\3049511425.py:9: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
620816it [02:08, 4818.22it/s]

2023-04-02 19:08:25,478 Dictionary created for label 'class' with 4 values: 0 (seen 308976 times), 1 (seen 248183 times), 2 (seen 63657 times)


In [3]:
trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=5, learning_rate=0.05, mini_batch_size=64, monitor_test=True)

2023-04-02 19:30:25,757 ----------------------------------------------------------------------------------------------------
2023-04-02 19:30:25,757 Model: "TextClassifier(
  (embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=4

100%|██████████| 1213/1213 [07:13<00:00,  2.80it/s] 

2023-04-02 20:33:15,155 Evaluating as a multi-label problem: False


2023-04-02 20:33:15,420 DEV : loss 0.9240173697471619 - f1-score (micro avg)  0.5679


100%|██████████| 1213/1213 [07:14<00:00,  2.79it/s] 


2023-04-02 20:40:58,412 Evaluating as a multi-label problem: False
2023-04-02 20:40:58,667 TEST : loss 0.923307478427887 - f1-score (micro avg)  0.5637
2023-04-02 20:41:36,653 BAD EPOCHS (no improvement): 0
2023-04-02 20:41:36,654 saving best model


NotImplementedError: 